In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
from spicy_snow.retrieval import retrieval_from_parameters
from spicy_snow.processing.s1_preprocessing import s1_dB_to_power

In [4]:
infp = Path('~/scratch/spicy/SnowEx-Data').expanduser()
outfp = Path('~/spicy-snow/SnowEx-Data-Hans').expanduser()
outfp.mkdir(exist_ok=True)

# param_stats = xr.open_dataset('/bsuhome/zacharykeskinen/scratch/spicy/param_npys/param_stats.nc')

In [9]:
# a = 2.5
# b = 0.2
# c = 0.55

a, b, c = [2, 0.5, 0.44]

for fp in infp.glob('*.nc'):
    # if fp.stem != 'Banner_2021-03-15':
    #     continue
    # sub_stats = param_stats.sel(location = fp.stem)
    # c_best = sub_stats.sel(C = sub_stats['mae'].idxmin('C'))
    # a_c_best = c_best.sel(A = c_best['pearsonr'].idxmax('A'))
    # best = a_c_best.sel(B = a_c_best['pearsonr'].idxmax('B'))
    # a, b, c = best.A.mean(), best.B.mean(), best.C.mean()
    
    ds = xr.open_dataset(fp)
    new_ds = retrieval_from_parameters(ds, A = a, B = b, C = c, freezing_snow_thresh = 1)
    # new_ds = add_confidence_angle(new_ds)
    if fp.stem == 'Banner_2021-03-15':
        new_ds = new_ds.isel(time = slice(12, None))
    new_ds.attrs['param_A'] = a
    new_ds.attrs['param_B'] = b
    new_ds.attrs['param_C'] = c
    new_ds['inc'] = new_ds.sel(band = 'inc')['s1'].mean(dim = 'time')
    new_ds[['snow_depth', 'wet_snow', 'ims', 'fcf', 'lidar-sd', 'lidar-vh', 'lidar-dem', 'inc']].to_netcdf(outfp.joinpath(fp.name))
    # new_ds.time.to_dataframe().to_csv(outfp.joinpath('times', fp.stem + '.csv'))